# Text Classification with AG's News Topics - Ludwig CLI

*We recommend using a GPU runtime for this example. In the Colab menu bar, choose **Runtime** > **Change Runtime Type** and choose **GPU** under Hardware Accelerator.*

In this notebook, we will show how to use the Ludwig CLI to:


1.   [Download a Dataset](#scrollTo=Download_Dataset)
2.   [Train a Ludwig Model](#scrollTo=Train)
3.   [Evaluate the trained model](#scrollTo=Evaluate)
4.   [Visualize training and test metrics](#scrollTo=Visualize_Metrics)
5.   [Make predictions on New Data](#scrollTo=Make_Predictions_on_New_Data)

In [1]:
# Prerequisite: Installs the latest version of Ludwig in the Colab environment
!python -m pip install git+https://github.com/ludwig-ai/ludwig.git --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 134 kB 3.1 MB/s 
     |████████████████████████████████| 332 kB 16.9 MB/s 
     |████████████████████████████████| 47 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
     |████████████████████████████████| 144 kB 50.1 MB/s 
     |████████████████████████████████| 271 kB 45.9 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 


# Download Dataset

We'll be using AG's news topic classification dataset, a common benchmark dataset for text classification. This dataset is a subset of the full AG news dataset, constructed by choosing 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.


This dataset contains four columns:

| column      | description                                                |
|-------------|------------------------------------------------------------|
| class_index | integer 1-4 corresponding to "world", "sports", "business", "sci_tech" respectively |
| class       | The topic label, one of "world", "sports", "business", "sci_tech" |
| title       | Title of the news article                                  |
| description | Description of the news article                            |


In [2]:
# Downloads the AG news dataset to the current working directory.
!ludwig datasets download agnews

NumExpr defaulting to 2 threads.
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Datasets download



# Train

## Define ludwig config

The Ludwig config declares the machine learning task. It tells Ludwig what to predict, what columns to use as input, and optionally specifies the model type and hyperparameters.

Here, for simplicity, we'll try to predict **class** from **title**.

In [3]:
config_yaml = """
input_features:
  -
    name: title
    type: text
    level: word
    encoder: parallel_cnn
output_features:
  -
    name: class
    type: category
preprocessing:
  force_split: true
  split_probabilities: [0.7, 0.1, 0.2]
trainer:
  epochs: 3
"""

# Writes config to a file called "config.yaml"
with open("config.yaml", "w") as f:
  f.write(config_yaml)

## Create and train a model

In [4]:
# Trains the model. This cell might take a few minutes.
!ludwig train --dataset agnews.csv -c config.yaml

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Train


╒════════════════════════╕
│ EXPERIMENT DESCRIPTION │
╘════════════════════════╛

╒══════════════════╤═══════════════════════════════════════════════════════════════════╕
│ Experiment name  │ experiment                                                        │
├──────────────────┼───────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                               │
├──────────────────┼───────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/experiment_run                                   │
├──────────────────┼───────────────────────────────────────────────────────────────────┤
│ ludwig_vers

# Evaluate

In [5]:
# Generates predictions and performance statistics for the test set.
!ludwig evaluate --model_path results/experiment_run/model \
                 --dataset agnews.csv \
                 --split test \
                 --output_directory test_results

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Evaluate

Dataset path: agnews.csv
Model path: results/experiment_run/model

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)
Loading metadata from: results/experiment_run/model/training_set_metadata.json
Evaluation: 100% 199/199 [00:03<00:00, 62.70it/s]

===== class =====
accuracy: 0.8729909062385559
hits_at_k: 0.9884072542190552
loss: 0.3707893490791321
overall_stats: { 'avg_f1_score_macro': 0.8725554588311559,
  'avg_f1_score_micro': 0.872990922309113,

# Visualize Metrics

In [6]:
# Visualizes confusion matrix, which gives an overview of classifier performance
# for each class.
!ludwig visualize --visualization confusion_matrix \
                  --ground_truth_metadata results/experiment_run/model/training_set_metadata.json \
                  --test_statistics test_results/test_statistics.json \
                  --output_directory visualizations \
                  --file_format png


# If you run ludwig visualize locally, visualizations will automatically show in
# a window. Here in Colab, we can run the following code to load and display
# generated plots inline.
from IPython import display
import ipywidgets
from pathlib import Path

ipywidgets.HBox([
  ipywidgets.Image(value=Path("visualizations/confusion_matrix__class_top5.png").read_bytes()),
  ipywidgets.Image(value=Path("visualizations/confusion_matrix_entropy__class_top5.png").read_bytes()),
])

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'


In [7]:
# Visualizes learning curves, which show how performance metrics changed over
# time during training.
!ludwig visualize --visualization learning_curves \
                  --ground_truth_metadata results/experiment_run/model/training_set_metadata.json \
                  --training_statistics results/experiment_run/training_statistics.json \
                  --file_format png \
                  --output_directory visualizations


# If you run ludwig visualize locally, visualizations will automatically show in
# a window. Here in Colab, we can run the following code to load and display
# generated plots inline.
from IPython import display
import ipywidgets
from pathlib import Path

ipywidgets.VBox([
  ipywidgets.HBox([
    ipywidgets.Image(value=Path("visualizations/learning_curves_combined_loss.png").read_bytes()),
    ipywidgets.Image(value=Path("visualizations/learning_curves_class_loss.png").read_bytes()),
  ]),
  ipywidgets.HBox([
    ipywidgets.Image(value=Path("visualizations/learning_curves_class_accuracy.png").read_bytes()),
    ipywidgets.Image(value=Path("visualizations/learning_curves_class_hits_at_k.png").read_bytes())
  ]),
])

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'


# Make Predictions on New Data

Lastly we'll show how to generate predictions for new data.

The following are some recent news headlines. Feel free to edit or add your own strings to text_to_predict to see how the newly trained model classifies them.

In [8]:
import pandas as pd

text_to_predict = pd.DataFrame({
  "title": [
    "Google may spur cloud cybersecurity M&A with $5.4B Mandiant buy",
    "Europe struggles to meet mounting needs of Ukraine's fleeing millions",
    "How the pandemic housing market spurred buyer's remorse across America",
  ]
})

text_to_predict.to_csv('text_to_predict.csv', index=False)

In [9]:
!ludwig predict --model_path results/experiment_run/model \
                --dataset text_to_predict.csv \
                --output_directory predictions

# Loads predictions
predictions = pd.read_parquet('predictions/predictions.parquet')
predictions

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Predict

Dataset path: text_to_predict.csv
Model path: results/experiment_run/model

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)
Loading metadata from: results/experiment_run/model/training_set_metadata.json
Prediction: 100% 1/1 [00:00<00:00, 131.90it/s]
Saved to: predictions


,class_predictions,class_probabilities,class_probability,class_probabilities_<UNK>,class_probabilities_sci_tech,class_probabilities_sports,class_probabilities_world,class_probabilities_business
0,sci_tech,"[4.670132613959765e-10, 0.9424473643302917, 5....",0.942447,4.670133e-10,0.942447,0.000053,0.001889,0.055610
1,world,"[3.0950943141760945e-08, 0.003253281582146883,...",0.993360,3.095094e-08,0.003253,0.000704,0.993360,0.002682
2,business,"[1.1677035445245565e-06, 0.39406904578208923, ...",0.531147,1.167704e-06,0.394069,0.000373,0.074410,0.531147
